## EasyLabel!

In am going to start by generating automatically the file config.xml which contains the configuration that the Label 
Studio program will consider.

In [1]:
import xml.etree.ElementTree as ET

## This function takes as input the name of the different classes
## It creates the file config.xml in the output directory.
## list_of_labels is a list of strings
def creat_config_xml_image_classification(list_of_labels):
    # I am going to start by sorting list_of_labels
    list_of_labels.sort()
    tree = ET.parse('./configuration/prepare_conf.xml')
    root = tree.getroot()
    Choices = root.getchildren()[1]
    for label in list_of_labels:
        data = ET.Element("Choice", {"value": label})
        Choices.append(data)        
    tree.write('config.xml')

In [2]:
list_of_labels = ["Back","Discard","Front","Left","Right"]

In [3]:
creat_config_xml_image_classification(list_of_labels)

/c/Users/yfakihan/workspace/POC/POC1/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  # This is added back by InteractiveShellApp.init_path()


So we get a file named config.xml that we will use it to configure label studio.

In this classification problem, the number of classes is:

In [4]:
num_class = len(list_of_labels)
print(num_class)

5


Nos let’s create the Keras model that will use the Neural Architecture Search Network (NASNet) models:
There are two main types of models available in Keras: The sequential model, and the Model class used with the functional API. 

In the functional API, given some input tensor(s) and output tensor(s), the model will include all layers required to do the computation.

And in the sequential model, the model is a linear stack of layers. We can create a Sequential model by passing a list of layers to the constructor. 

In our classification problem, since our model will not have a very difficult architecture and specially that it will use an already pre-trained model, I am going to use the Sequential model because it seems to be well adapted to this problem.  

Furthermore, to better solve this problem, I am going to use the transfer learning method that focuses on storing knowledge gained from a problem met in the past and adapt it to the actual one.

In our case, we use the NASNet model, with weights pre-trained on ImageNet. 


In [5]:
from keras.models import Sequential
from keras.applications.nasnet import NASNetMobile
from keras.layers import Dense
from keras.optimizers import SGD

def creat_compiled_model():
    model = Sequential()
    model.add(NASNetMobile(input_shape=(224, 224, 3), include_top=False,
                           weights='imagenet', pooling='avg'))
    model.add(Dense(num_class,activation='softmax'))
    model.layers[0].trainable = True
    model.layers[1].trainable = True
    sgd = SGD(learning_rate=0.01, momentum=0.01, nesterov=False)
    model.compile(optimizer=sgd, loss='hinge',  metrics=['accuracy'])
    return model    

Using TensorFlow backend.


The function above creates the model that we want to fit. I started by creating an instance of the class sequential. Then, I added two layers to this instance, the first one is the NASNet Mobile model already pre-trained on the Imagenet data set. The second layer is the one representing the output of this model.

Finally, to configure the model for training, I make use of the compile function. To that purpose, I choose the Stochastic Gradient Descent as an optimizer, the Hinge loss as a loss function and finally the accuracy as a metric to be evaluated by the model during training and testing. 


In [6]:
model = creat_compiled_model()

From now on, the variable model represents our model. And it is ready to be trained. 

Of course, the configuration/choice of parameters of the different methods used in the function creat_compiled_model is little bit arbitrary. 
At this point of this project, I can not say what is the best configuration that I must use, but later I am going to do a lot of tests to choose the best combination of parameters. For example, in the future I may use the open source hyperparameter optimization framework ‘Optuna’ to choose these parameters. 

As we did in the notebook named keras_tensorflow, we can use the result exported from Label Studio representing the labels of the elements of our data set to fit our model, so we will be able to predict the label of some new elements.

But this time we are going to do better 😉 ! 

Indeed, we are going to use the option called Machine Learning backend of Label Studio to do what they call the “Pre-labeling” and the “Online Learning”. Thanks to these two options which are very similar, we can use our model predictions for pre-labeling and to simultaneously update (retrain) our model while new annotations are coming. So, at the end of the labeling task our model will be already trained and ready! 